In [2]:
import tensorflow as tf

In [3]:
class MultiHeadAttention(tf.keras.layers.Layer):

  def __init__(self, d_model, num_heads, name="multi_head_attention"): # 정의하기
    super(MultiHeadAttention, self).__init__(name=name)
    self.num_heads = num_heads # 8
    self.d_model = d_model # 512

    assert d_model % self.num_heads == 0

    self.depth = d_model // self.num_heads

    self.query_dense = tf.keras.layers.Dense(units=d_model) #WQ
    self.key_dense = tf.keras.layers.Dense(units=d_model) #WK
    self.value_dense = tf.keras.layers.Dense(units=d_model) #WV

    self.dense = tf.keras.layers.Dense(units=d_model) #WO

  def split_heads(self, inputs, batch_size): # 아래의 call 함수에서 헤드를 나누기 위해서 호출
    inputs = tf.reshape(
        inputs, shape=(batch_size, -1, self.num_heads, self.depth))
    return tf.transpose(inputs, perm=[0, 2, 1, 3])

  def call(self, inputs):
    query, key, value, mask = inputs['query'], inputs['key'], inputs[
        'value'], inputs['mask']
    batch_size = tf.shape(query)[0]

    # 1. WQ, WK, WV에 해당하는 밀집층 지나기
    query = self.query_dense(query) # (batch_size, seq_len, d_model) 
    key = self.key_dense(key) # (batch_size, seq_len, d_model)
    value = self.value_dense(value) # (batch_size, seq_len, d_model)

    # 2. 헤드 나누기 (split_heads의 transpose에 의해 shape이 결정됨)
    query = self.split_heads(query, batch_size) # (batch_size, num_heads, seq_len, d_model/num_heads) 이것이 결과 shape
    key = self.split_heads(key, batch_size) # (batch_size, num_heads, seq_len, d_model/num_heads)
    value = self.split_heads(value, batch_size) # (batch_size, num_heads, seq_len, d_model/num_heads)

    # 3. 스케일드 닷 프로덕트 어텐션. 앞서 구현한 함수 사용.
    scaled_attention = scaled_dot_product_attention(query, key, value, mask)
    scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])

    # 4. 헤드 연결(concatenate)하기
    concat_attention = tf.reshape(scaled_attention,
                                  (batch_size, -1, self.d_model))

    # 5. WO에 해당하는 밀집층 지나기
    outputs = self.dense(concat_attention)

    return outputs # 최종 결과 리턴